# 3 – Analysis of the Altered Dataset (`altered.json`)

## Table of Contents
1. [Introduction](#introduction)
2. [Setup and Data Loading](#setup-and-data-loading)
3. [3a. Merging Previously Merged Data with Altered Data](#3a-merge-altered)
4. [3b. Elevators with More Than 5 Alterations](#3b-more-than-5)
5. [3c. Different Combinations of Alteration Type & Status](#3c-alteration-combinations)
6. [Conclusion](#conclusion)


<a id="introduction"></a>
## 1. Introduction

In this third part of our ETL project, we focus on the **Altered** dataset (`altered.json`). We will:

- Merge the previously merged License+Installed data with the Altered dataset, keeping all elevators **even if** they have no alterations.
- Identify elevators that have **more than 5** alteration records and display them.
- Determine different combinations of “Alteration Type” and “Status of Alteration Request” and **count** each combination.

This sets the stage for further analysis of how modifications have been requested and handled over time.


<a id="setup-and-data-loading"></a>
## 2. Setup and Data Loading


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set(style="whitegrid")
%matplotlib inline

# Step 1: Check the current working directory
print("Current working directory:", os.getcwd())

# Step 2: List available files in the directory
print("Files in the current directory:", os.listdir())

# Step 3: Load the previously merged data from part 2 
# (License + Installed) 
try:
    merged_installed_df = pd.read_csv('merged_installed_data.csv')
    print(f"Shape of merged_installed_df: {merged_installed_df.shape}")
except FileNotFoundError as e:
    print("Error: The file 'merged_installed_data.csv' was not found.")
    raise e

# Step 4: Load the Altered DataFrame
try:
    altered_df = pd.read_json('altered.json')
    print(f"Shape of altered_df: {altered_df.shape}")
except FileNotFoundError as e:
    print("Error: The file 'altered.json' was not found.")
    raise e

# Quick check of columns
print("\nColumns in merged_installed_df:", merged_installed_df.columns.tolist())
print("Columns in altered_df:", altered_df.columns.tolist())


Current working directory: /home/gamikarudev/programming/courses/codeboxx/ai-for-devs/AIC-103_Data_Science_Project/notebooks
Files in the current directory: ['.gitkeep', 'licence-ETL.ipynb', 'inspections-ETL.ipynb', '.ipynb_checkpoints', 'installed-ETL.ipynb', 'altered-ETL.ipynb']
Error: The file 'merged_installed_data.csv' was not found.


FileNotFoundError: [Errno 2] No such file or directory: 'merged_installed_data.csv'

<a id="3a-merge-altered"></a>
## 3. 3(a). Merging Previously Merged Data with Altered Data

**Requirement**:  
“Merge the previously merged dataset with the Altered dataset, **retaining** information even if elevators were not altered.”

**Objective**:  
- Perform a **left** merge from our installed Data (`merged_installed_df`) onto the `altered_df`. 
- This ensures we **keep all** rows from the installed dataset even if they do not appear in `altered_df`.


In [ ]:
# === 3(a) STEP 1: Standardize Key Column for the Altered DataFrame ===
# The JSON might call it "Elevating Devices Number" or "ElevatingDevicesNumber" etc.
# Let's rename for consistency:
altered_df.rename(columns={
    'Elevating Devices Number': 'ElevatingDevicesNumber',
    'Alteration  Location': 'AlterationLocation'
}, inplace=True)

# Print columns again to confirm
print("Altered DF columns after rename:", altered_df.columns.tolist())


In [ ]:
# === 3(a) STEP 2: Merge DataFrames ===
# We want a left merge on merged_installed_df to keep all those rows,
# even if there's no match in altered_df.
key_col = 'ElevatingDevicesNumber'

merged_altered_df = pd.merge(
    merged_installed_df, altered_df,
    on=key_col, how='left'
)

print("Number of rows in merged_installed_df:", len(merged_installed_df))
print("Number of rows in altered_df:", len(altered_df))
print("Number of rows in merged_altered_df (left join):", len(merged_altered_df))

# Optional: how many are left unmatched
unmatched_count = merged_altered_df[ 'originating service request number' ].isnull().sum()
print(f"\nRows with no alteration info: {unmatched_count} (this is expected).")

# Quick peek
display(merged_altered_df.head(5))



### Data Quality & Additional Observations

- We see **21237** unmatched rows after the left join (where `originating service request number` is `NaN`). That’s expected because not all elevators are altered.  
- For those that do have alterations, we see each row references a specific **service request number**, a short **Summary** (like “Minor B Alteration Machine Guarding”), and a **Status**.  
- **Location** often includes city, postal code, etc. If we wanted, we could cross‐reference it with the location from the installed dataset to see if any addresses mismatch.  

**Potential Next Steps**  
1. **Analyze cost or date** of these alterations if cost or date is present or can be derived. Are repeated modifications happening in older buildings, certain city neighborhoods, or with a particular contractor?  
2. **Dig into “Rejected”** or “Pending Follow Up” statuses to see if certain building owners are struggling with consistent compliance.  

---



<a id="3b-more-than-5"></a>
## 4. 3(b). Elevators with More Than 5 Alterations

**Requirement**:  
“Identify elevators with **more than 5** alterations and display all rows related to those elevators.”

**Approach**:
1. Count the number of Alteration records **per elevator** in `merged_altered_df`.
2. Filter for those with a count > 5.
3. Display all the relevant rows for those elevators.


In [ ]:
# === 3(b) STEP 1: Group by Elevator & Count Alteration Rows ===
# The 'originating service request number' might identify unique alteration rows.
# Or we can just count rows.

grouped = merged_altered_df.groupby(key_col)['originating service request number'].count()

# Elevators with more than 5 alterations:
elevators_more_than_5 = grouped[grouped > 5].index.tolist()
print("Elevators with more than 5 alterations:", elevators_more_than_5)

# === 3(b) STEP 2: Display all rows for these elevators in merged_altered_df ===
if len(elevators_more_than_5) > 0:
    more_than_5_df = merged_altered_df[merged_altered_df[key_col].isin(elevators_more_than_5)].copy()
    print(f"\nTotal rows for these {len(elevators_more_than_5)} elevators: {len(more_than_5_df)}")
    display(more_than_5_df.sort_values(by=key_col))
else:
    print("\nNo elevator has more than 5 alterations.")


### Table of Rows for Elevators With >5 Alterations Interpretation

### Meaning & Insights

1. **Elevator with Number 14191**  
   - Has multiple Minor B alterations. Some are “Closed,” one is “Design Registered,” and one is “Complete.” This suggests repeated smaller‐scope modifications, possibly an ongoing maintenance or modernization cycle.

2. **Elevator with Number 62798**  
   - Also multiple Minor alterations (some B, some A).  
   - Many with status “Passed.” Possibly a building that invests heavily in consistent upgrades.

3. **Significance**:  
   - We’re seeing that certain buildings or elevator owners handle *frequent* alterations (5+ times). This might correlate with usage, building age, or occupant turnover.  
   - Could prompt deeper questions: Are these repeated small changes cost‐effective? Do frequent modifications relate to building occupant demands?

By listing all rows for these highly modified elevators, we can investigate patterns in location, owners, or reasons behind repeated changes.

---


<a id="3c-alteration-combinations"></a>
## 5. 3(c). Different Combinations of “Alteration Type” & “Status of Alteration Request”

**Requirement**:  
“Determine and count the different combinations of `Alteration Type` and `Status of Alteration Request`.”

**Objective**:  
- We can do a simple `groupby` or `pd.crosstab` to see how many times each `(Alteration Type, Status)` pair occurs.


**Optional**: We could do a pivot or crosstab for a more matrix‐style view:



In [ ]:
combination_crosstab = pd.crosstab(merged_altered_df['Alteration Type'], merged_altered_df['Status of Alteration Request'])
print("\nCrosstab of Alteration Type vs Status:")
display(combination_crosstab)

Below is a short **interpretation** and **discussion** of the **crosstab** and **sample rows**, explaining what these results might mean and highlighting potential insights. It should help any non‐technical stakeholders make sense of the data and see where further exploration might be useful.

---

### Crosstab Interpretation

#### What Does Each Column Mean?

- **Alteration Type**:  
  - **ED-Major Alteration** typically indicates a significant upgrade or modernization.  
  - **ED-Minor A Alteration** and **ED-Minor B Alteration** are smaller‐scope modifications. (We might infer “Minor A” vs. “Minor B” has different code designations, possibly meaning different extents or safety impacts.)

- **Status of Alteration Request** (each column):
  - **Passed** generally means the alteration request was completed and passed inspection or final sign‐off.
  - **Closed** could mean the record has been closed out, though not always “passed” (perhaps the project ended differently).
  - **Complete** might represent partial or full completion. 
  - **Design Registered**, **Conditionally Registered**, or **On Hold** are likely statuses within elevator production and alteration  workflow. 
  - **Pending Follow Up** or **Rejected** means the process wasn’t fully accepted.

#### Observing Patterns

- **ED-Minor A Alteration** has **13,080** “Passed” records, which is by far the largest single combination. This indicates that the **most common** scenario in the dataset is a Minor A alteration that successfully concluded (passed).
- **ED-Major Alteration** has **6,370** “Passed” records, also significant but about half as many as Minor A. We also see 392 are “Pending Follow Up,” meaning there’s still some action required as of the data captured in the data sets. 
- **ED-Minor B Alteration** has **7,086** “Passed” records. Interestingly, it also has **454** “Design Registered”—which means these Minor B requests are still in some form of design or registration stage more often than for Minor A.

##### Key Takeaways
1. **Most Common Status**: “Passed” across all three alteration types, making up the bulk of concluded alterations.
2. **Design Registrations** Appear More Common in Minor B Alterations than A or Major (454 vs. 63 or 79). Possibly Minor B tasks often require design registration steps.
3. **Possible High Pending Follow‐Ups**: Notably, Minor A has **1,058** “Pending Follow Up,” which might indicate a backlog or slower finalization process, since it's more likely that most elevators will have minor alterations required more often than severe or major alterations. 

These patterns can help us prioritize resources or investigate further why Minor A has so many follow‐ups, or Minor B has a large number of design registrations.

---



<a id="conclusion"></a>
## Conclusion

In **Part 3** of our ETL project, we merged the combined (License + Installed) dataset with the Altered dataset, ensuring we retained every elevator—even those without any alterations. Using these unified records, we identified and displayed elevators having more than five alterations, providing a clear view of assets that undergo unusually frequent modifications. We also compiled a summary of “Alteration Type” paired with “Status of Alteration Request,” revealing that:

- **Minor A Alterations** dominate the dataset, most commonly concluding with a “Passed” status, yet also showing a high number of “Pending Follow Up” records.  
- **Major** and **Minor B Alterations** are less frequent but still exhibit a significant portion marked as “Passed,” reflecting successfully completed modifications.  
- A small subset of elevators have a disproportionately high count of alterations (5+), indicating these assets undergo repeated or complex modification cycles.

This analysis provides a **springboard** for deeper questions:
- What drives multiple smaller alterations versus fewer major overhauls?
- Are repeated alterations correlated with older infrastructure, specific locations, or particular building owners?
- How do different alteration statuses track over time, and what cost or compliance implications arise from “Pending Follow Up” or “Rejected” statuses?

By unifying and exploring the Altered dataset in tandem with License and Installed data, we’ve laid the groundwork for further insights into how elevator modifications impact operational reliability and cost. **Future steps** may include integrating Inspection data for an end‐to‐end picture of each elevator’s lifecycle, investigating cost patterns, and establishing predictive models to optimize maintenance strategies and safety outcomes.


In [ ]:
merged_altered_df.to_csv('merged_altered_data.csv', index=False)
